In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import warnings

from utils import make_dataset

warnings.filterwarnings('ignore')

TARGET_COLS = 'two_year_recid'
COMPAS_SCORES_COLS = 'decile_score'
NUMERICAL_FEATURE_COLS = ['age',
                          'juv_fel_count','juv_misd_count','juv_other_count',
                          'priors_count','jail_time']
CATEGORICAL_FEATURE_COLS = ['sex','race',
                            'c_charge_degree']
FEATURE_NAMES = NUMERICAL_FEATURE_COLS+CATEGORICAL_FEATURE_COLS

PROTECTED_COLS = ['sex','race']

data = pd.read_csv('../data/processed/compas-scores-two-years-processed.csv')
data.head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,jail_time,sex,race,c_charge_degree,decile_score,two_year_recid,compas_score
0,69,0,0,0,0,0.0,1,1,1,1,0,0.1
1,34,0,0,0,0,10.0,1,2,1,3,1,0.3
2,24,0,0,1,4,1.0,1,2,1,4,1,0.4
3,23,0,1,0,1,0.0,1,2,1,8,0,0.8
4,43,0,0,0,2,0.0,1,1,1,1,0,0.1


In [2]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=1234)

X_train, y_train = train[FEATURE_NAMES], train[TARGET_COLS]
X_test, y_test = test[FEATURE_NAMES], test[TARGET_COLS]

In [3]:
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=1234)
clf.fit(X_train, y_train)

y_test_pred = clf.predict_proba(X_test)
test['recid_prediction'] = (y_test_pred[:,1] >0.5).astype(int)

roc_auc = roc_auc_score(y_test, y_test_pred[:,1])

In [4]:
from aif360.metrics.classification_metric import ClassificationMetric

from utils import make_dataset

bias_info = {'favorable_label':0,
             'unfavorable_label':1,
             'protected_columns':['race'],
            }
privileged_info = {'unprivileged_groups':[{'race': 2},
                                          {'race': 1},
                                          {'race': 4},
                                          {'race': 5},
                                          {'race': 6}],
                   'privileged_groups':[{'race': 3}]
                  }


ground_truth_test = make_dataset(test, 'two_year_recid', **bias_info, **privileged_info)

prediction_test = make_dataset(test, 'recid_prediction', **bias_info, **privileged_info)

clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**privileged_info)

In [5]:
import joblib
from utils import display_results

joblib.dump((clf_metric,roc_auc), '../results/0.2-baseline_model.pkl')
display_results('../results/0.2-baseline_model.pkl')

,metric_names,scores
0,roc_auc_score,0.728357
1,true_positive_rate_difference,-0.102441
2,false_positive_rate_difference,-0.247320
3,false_omission_rate_difference,-0.035370
4,false_discovery_rate_difference,-0.042279
5,error_rate_difference,-0.034496
6,false_positive_rate_ratio,0.621169
7,false_negative_rate_ratio,1.714525
8,false_omission_rate_ratio,0.905385
9,false_discovery_rate_ratio,0.877524


In [7]:
display_results('../results/0.1-compas_model.pkl')

,metric_names,scores
0,roc_auc_score,0.701245
1,true_positive_rate_difference,0.140105
2,false_positive_rate_difference,0.184571
3,false_omission_rate_difference,0.009135
4,false_discovery_rate_difference,-0.028397
5,error_rate_difference,-0.011407
6,false_positive_rate_ratio,1.445276
7,false_negative_rate_ratio,0.834367
8,false_omission_rate_ratio,1.013509
9,false_discovery_rate_ratio,0.956340


## Questions

* Is our baseline model better than compas model?
* Is it one model better on all metrics?